In [29]:
##KENDALL CORR

###DATASET STROKE
import pandas as pd
import numpy as np
from numpy import nan as NA
#data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv')
data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv') #Dataset con outliers
new_data= data.fillna(method='ffill')
new_data2= pd.get_dummies(new_data, columns= ["gender"])
new_data3= pd.get_dummies(new_data2, columns= ["ever_married"])
new_data4= pd.get_dummies(new_data3, columns= ["work_type"])
new_data5= pd.get_dummies(new_data4, columns= ["Residence_type"])
new_data6= pd.get_dummies(new_data5, columns= ["smoking_status"])
new_data6= new_data6.drop([ 'Residence_type_Urban', 'work_type_children', 'Residence_type_Rural','work_type_Self-employed', 'work_type_Private','work_type_Never_worked', 'work_type_Govt_job','ever_married_Yes','ever_married_No','gender_Other', 'gender_Male','gender_Female'], axis=1)
new_data6=new_data6.drop(columns='id')


## BALANCEAR DATOS
x=new_data6.loc[new_data6['stroke']==0]
m = x.sample(249)
y=new_data6.loc[new_data6['stroke']==1]
new_data6 = pd.concat([y,m],axis=0)

##FEATURES STROKE

features=new_data6.drop(columns='stroke')
target = new_data6.iloc[:,2]

##MATRIZ DE COVARIANZA ROBUSTA MÉTODOS
from statistics import stdev
import numpy as np
from scipy.stats import spearmanr
from statsmodels import robust
from scipy.stats import kendalltau
from scipy import stats

def covariance(x,y):
  coef, p = kendalltau(x,y)
  VI = coef*(stdev(x)*stdev(y)) #Existe la opción de hacerlo con el MAD para que sea más robusto,
  #Sin embargo, con nuestros datos esto resulta en una matriz singular (no invertible)
  return VI

def covarianceM(x):
  covar = np.zeros((len(x.columns),len(x.columns)), dtype=float)
  for i,j in np.ndindex(covar.shape): 
      VI=covariance(x.iloc[:,i],x.iloc[:,j])
      covar[i,j] = VI
  return covar

##BETAS 

def betas(features, target):
  cova = covarianceM(features)
  Cxinv = np.linalg.inv(cova)
  covars = np.zeros((len(features.columns)), dtype=float)
  for i in range(len(features.columns)):
    covars[i] = covariance(features.iloc[:,i],target)
  Betas=np.dot(Cxinv,(covars.T))

##BETA 0
  promedio = target.mean()
  Beta0 = promedio - np.dot(features.mean(axis=0),Betas)
  return Betas, Beta0

#Link function o función sigmoide
def sigmoid(betas, beta0, features):
    Xbetas = np.dot(features,betas)
    y = beta0 + Xbetas
    return 1 / (1 + np.exp(-y))
Betas, Beta0 = betas(features,target)

##Prueba
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

auc = []
#for i in range(1000):
x_train, x_test, y_train, y_test=train_test_split(features,target)

Betas, Beta0 = betas(x_train,y_train)
Prob = sigmoid(Betas, Beta0, x_test) 
Prediction = np.zeros((len(x_test)), dtype=float)
for i in range(len(x_test)):
  if Prob[i]>=0.5:
    Prediction[i]=1.0
  else:
    Prediction[i]=0.0

fpr, tpr, threshold = metrics.roc_curve(y_test,Prediction)
roc_auc = metrics.auc(fpr, tpr)
auc.append(roc_auc)
acc = accuracy_score(y_test, Prediction)
print(confusion_matrix(y_test,Prediction))
print(acc)
print(auc)


[[92 17]
 [ 0 16]]
0.864
[0.9220183486238532]


          age  hypertension  ...  smoking_status_never smoked  smoking_status_smokes
0        67.0             0  ...                            0                      0
1        61.0             0  ...                            1                      0
2        80.0             0  ...                            1                      0
3        49.0             0  ...                            0                      1
4     46445.0             1  ...                            1                      0
...       ...           ...  ...                          ...                    ...
341      37.0             0  ...                            0                      0
2153     54.0             0  ...                            0                      0
421      12.0             0  ...                            0                      0
2625     32.0             0  ...                            0                      0
422      63.0             0  ...                            0    

In [30]:
##NO-ROBUSTA

###DATASET STROKE
import pandas as pd
import numpy as np
from numpy import nan as NA
#data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv')
data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv') #Dataset con outliers
new_data= data.fillna(method='ffill')
new_data2= pd.get_dummies(new_data, columns= ["gender"])
new_data3= pd.get_dummies(new_data2, columns= ["ever_married"])
new_data4= pd.get_dummies(new_data3, columns= ["work_type"])
new_data5= pd.get_dummies(new_data4, columns= ["Residence_type"])
new_data6= pd.get_dummies(new_data5, columns= ["smoking_status"])
new_data6= new_data6.drop([ 'Residence_type_Urban', 'work_type_children', 'Residence_type_Rural','work_type_Self-employed', 'work_type_Private','work_type_Never_worked', 'work_type_Govt_job','ever_married_Yes','ever_married_No','gender_Other', 'gender_Male','gender_Female'], axis=1)
new_data6=new_data6.drop(columns='id')


## BALANCEAR DATOS
x=new_data6.loc[new_data6['stroke']==0]
m = x.sample(249)
y=new_data6.loc[new_data6['stroke']==1]
new_data6 = pd.concat([y,m],axis=0)

##FEATURES STROKE

features=new_data6.drop(columns='stroke')
target = new_data6.iloc[:,2]

##MATRIZ DE COVARIANZA ROBUSTA MÉTODOS
from statistics import stdev
import numpy as np
from scipy.stats import spearmanr
from statsmodels import robust
from scipy.stats import kendalltau
from scipy import stats

def covariance(x,y):
  coef, p = stats.pearsonr(x,y)
  VI = coef*(stdev(x)*stdev(y)) #Existe la opción de hacerlo con el MAD para que sea más robusto,
  #Sin embargo, con nuestros datos esto resulta en una matriz singular (no invertible)
  return VI

def covarianceM(x):
  covar = np.zeros((len(x.columns),len(x.columns)), dtype=float)
  for i,j in np.ndindex(covar.shape): 
      VI=covariance(x.iloc[:,i],x.iloc[:,j])
      covar[i,j] = VI
  return covar

##BETAS 

def betas(features, target):
  cova = covarianceM(features)
  Cxinv = np.linalg.inv(cova)
  covars = np.zeros((len(features.columns)), dtype=float)
  for i in range(len(features.columns)):
    covars[i] = covariance(features.iloc[:,i],target)
  Betas=np.dot(Cxinv,(covars.T))

##BETA 0
  promedio = target.mean()
  Beta0 = promedio - np.dot(features.mean(axis=0),Betas)
  return Betas, Beta0

#Link function o función sigmoide
def sigmoid(betas, beta0, features):
    Xbetas = np.dot(features,betas)
    y = beta0 + Xbetas
    return 1 / (1 + np.exp(-y))
Betas, Beta0 = betas(features,target)

##Prueba
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

auc = []
#for i in range(1000):
x_train, x_test, y_train, y_test=train_test_split(features,target)

Betas, Beta0 = betas(x_train,y_train)
Prob = sigmoid(Betas, Beta0, x_test) 
Prediction = np.zeros((len(x_test)), dtype=float)
for i in range(len(x_test)):
  if Prob[i]>=0.5:
    Prediction[i]=1.0
  else:
    Prediction[i]=0.0

fpr, tpr, threshold = metrics.roc_curve(y_test,Prediction)
roc_auc = metrics.auc(fpr, tpr)
auc.append(roc_auc)
acc = accuracy_score(y_test, Prediction)
print(confusion_matrix(y_test,Prediction))
print(acc)
print(auc)

[[74 38]
 [ 0 13]]
0.696
[0.8303571428571428]


In [37]:
##SPEARMAN CORR

###DATASET STROKE
import pandas as pd
import numpy as np
from numpy import nan as NA
#data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv')
data = pd.read_csv('https://raw.githubusercontent.com/asepulvede/ModelacionSimulacion4/main/healthcare-dataset-stroke-data.csv') #Dataset con outliers
new_data= data.fillna(method='ffill')
new_data2= pd.get_dummies(new_data, columns= ["gender"])
new_data3= pd.get_dummies(new_data2, columns= ["ever_married"])
new_data4= pd.get_dummies(new_data3, columns= ["work_type"])
new_data5= pd.get_dummies(new_data4, columns= ["Residence_type"])
new_data6= pd.get_dummies(new_data5, columns= ["smoking_status"])
new_data6= new_data6.drop([ 'Residence_type_Urban', 'work_type_children', 'Residence_type_Rural','work_type_Self-employed', 'work_type_Private','work_type_Never_worked', 'work_type_Govt_job','ever_married_Yes','ever_married_No','gender_Other', 'gender_Male','gender_Female'], axis=1)
new_data6=new_data6.drop(columns='id')


## BALANCEAR DATOS
x=new_data6.loc[new_data6['stroke']==0]
m = x.sample(249)
y=new_data6.loc[new_data6['stroke']==1]
new_data6 = pd.concat([y,m],axis=0)

##FEATURES STROKE

features=new_data6.drop(columns='stroke')
target = new_data6.iloc[:,2]

##MATRIZ DE COVARIANZA ROBUSTA MÉTODOS
from statistics import stdev
import numpy as np
from scipy.stats import spearmanr
from statsmodels import robust
from scipy.stats import kendalltau
from scipy import stats

def covariance(x,y):
  coef, p = spearmanr(x,y)
  VI = coef*(stdev(x)*stdev(y)) #Existe la opción de hacerlo con el MAD para que sea más robusto,
  #Sin embargo, con nuestros datos esto resulta en una matriz singular (no invertible)
  return VI

def covarianceM(x):
  covar = np.zeros((len(x.columns),len(x.columns)), dtype=float)
  for i,j in np.ndindex(covar.shape): 
      VI=covariance(x.iloc[:,i],x.iloc[:,j])
      covar[i,j] = VI
  return covar

##BETAS 

def betas(features, target):
  cova = covarianceM(features)
  Cxinv = np.linalg.inv(cova)
  covars = np.zeros((len(features.columns)), dtype=float)
  for i in range(len(features.columns)):
    covars[i] = covariance(features.iloc[:,i],target)
  Betas=np.dot(Cxinv,(covars.T))

##BETA 0
  promedio = target.mean()
  Beta0 = promedio - np.dot(features.mean(axis=0),Betas)
  return Betas, Beta0

#Link function o función sigmoide
def sigmoid(betas, beta0, features):
    Xbetas = np.dot(features,betas)
    y = beta0 + Xbetas
    return 1 / (1 + np.exp(-y))
Betas, Beta0 = betas(features,target)

##Prueba
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

auc = []
#for i in range(1000):
x_train, x_test, y_train, y_test=train_test_split(features,target)

Betas, Beta0 = betas(x_train,y_train)
Prob = sigmoid(Betas, Beta0, x_test) 
Prediction = np.zeros((len(x_test)), dtype=float)
for i in range(len(x_test)):
  if Prob[i]>=0.5:
    Prediction[i]=1.0
  else:
    Prediction[i]=0.0

fpr, tpr, threshold = metrics.roc_curve(y_test,Prediction)
roc_auc = metrics.auc(fpr, tpr)
auc.append(roc_auc)
acc = accuracy_score(y_test, Prediction)
print(confusion_matrix(y_test,Prediction))
print(acc)
print(auc)

[[59 59]
 [ 0  7]]
0.528
[0.75]
